# LSTM 尼采风格文本生成

TensorFlow 2.0 + Keras + LSTM 序列数据生成。完成**字符级**的尼采风格文本生成任务。

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.0.0'

## 采样策略

### Softmax Temperature (Softamx 温度)

Softmax Temperature：用于表示采样概率分布的熵，即表示所选择的下一个字符会有多么出人意料或多么可预测。更高的 temperature 得到的是熵更大的采样分布，会生成更加出人意料、更加无结构的生成数据，而更低的温度对应更小的随机性，以及更加可预测的生成数据。

- 更高的温度 = 更随机
- 更低的温度 = 更确定

## 准备数据

先获取语料

In [2]:
path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower() #转换为小写
print('Corpus length:', len(text))

Corpus length: 600893


In [3]:
# 提取的字符序列长度
maxlen = 60

# 每 step 个字符采样一个新序列
step = 3

# 保存提取的序列
sentences = []

# 保存目标，下一个字符
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# 语料唯一字符列表
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# 语料唯一字符列表索引
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# 向量化。one-hot 编码为二进制数组
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 58


## 模型定义

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars), activation='softmax'))

目标经过 one-hot 编码，所以用 categorical_crossentropy 作为损失。

In [5]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 训练语言模型并从中采样

给定一个训练好的模型和一个种子文本片段，可以通过重复以下操作来生成新的文本：

1. 给定目前已生成的文本，总模型中得到下一个字符的概率分布
2. 根据某个温度对分布进行重新加权
3. 根据重新加权后的分布对下一个字符进行随机采样
4. 将新字符添加到文本末尾

采样函数

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

文本生成循环。反复训练并生成文本，没轮过后都使用一系列不同的温度值来生成文本。

In [7]:
import random
import sys
        
def on_epoch_end(epoch, _):
    """
    函数在每次 epoch 训练后被调用
    """
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]: # 使用不同的采样温度生成
        print('----- diversity:', diversity)

        # 随机选择一个种子文本
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400): # 生成 400 个字符
            # 先对目前生成的字符进行 one-hot 编码
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            # 对下一个字符进行采样
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Train on 200278 samples
Epoch 1/60
 72192/200278 [=========>....................] - ETA: 3:47 - loss: 2.2984